This tutorial is for the calibration of an Xpad_flat on the Soleil Diffabs Diffractometer.

The calibration data is a scan on the delta motor with a CeO2 calibrant.

In [1]:
# first the imports
from typing import Iterator, List, NamedTuple, Text, Tuple

import os
import functools

from collections import namedtuple

import numpy
import pylab
import pyFAI

from h5py import Dataset, File

from pyFAI.gui import jupyter

# local import
from soleil import *

In [2]:
# temporary until the ruch is ON
ROOT = os.path.join("/home", "experiences", "instrumentation", "picca", "data", "99160066", "2016", "Run2")
PUBLISHED = os.path.join("/home", "experiences", "instrumentation", "picca", "data", "99160066", "published-data")

In [3]:
# here the integration with the previously generated calibration file.
def integrate(json: str) -> None:
    """Integrate a file with a json calibration file"""
    filename = os.path.join(ROOT, "scan_77_01.nxs")
    gonio = pyFAI.goniometer.Goniometer.sload(json)
    wavelength = 4.85945727522e-11
    multicalib = MultiCalib(os.path.join(ROOT, "scan_4_01.nxs"),
                            MetaDataSource("",
                                           H5PathContains("scan_data/actuator_1_1"),
                                           H5PathOptionalItemValue("MARS/D03-1-CX0__DT__DTC_2D-MT_Tz__#1/raw_value", -1.0)),
                            [], "LaB6", "xpad_flat", wavelength)

    with h5py.File(filename, mode='r') as h5file:
        images = []
        positions = []
        for metadata in gen_metadata(h5file, multicalib):
            images.append(metadata.img)
            positions.append((metadata.tx, metadata.tz))
        mai = gonio.get_mg(positions)
        res = mai.integrate1d(images, 10000)
        jupyter.plot1d(res)
        pylab.show()

In [4]:
# save all the ref as images in order to do the calibration with
# pyFAI-calib[2].

def ExportEdf(params: Calibration):
    save_as_edf(params, PUBLISHED)

In [5]:
# do the calibration
json = os.path.join(PUBLISHED, "calibration", "calibration.json")

distance = Length(0.258705917299)
poni1_scale = 0.001
poni1_offset = Length(0.132825374721)
poni2_scale = 0.0012272727272727272
poni2_offset = Length(-0.9488181818181818)
rot1 = Angle(0.00388272369359)
rot2 = Angle(-0.00942588451226)
rot3 = Angle(7.19961198098e-07)

params = Calibration(os.path.join(PUBLISHED, "calibration", "XRD18keV_26.nxs"),
                     DatasetPathWithAttribute("interpretation", b"image"),
                     DatasetPathContains("scan_data/actuator_1_1"),
                     [2, 5, 8],
                     Calibrant("CeO2"),
                     Detector("Xpad_flat"),
                     Wavelength(6.8877e-11),
                     CalibrationParameters(Parameter[Length]("dist", distance, (distance, distance)),
                                           Parameter[Length]("poni1_offset", poni1_offset, (0, 0.2)),
                                           Parameter[float]("poni1_scale", poni1_scale, (0, 0.002)),
                                           Parameter[Length]("poni2_offset", poni2_offset, (-1, -0.7)),
                                           Parameter[float]("poni2_scale", poni2_scale, (-1, 1)),
                                           Parameter[Angle]("rot1", rot1, (rot1, rot1)),
                                           Parameter[Angle]("rot2", rot2, (rot2, rot2)),
                                           Parameter[Angle]("rot3", rot3, (rot3, rot3))
                                          )
                    )

calibration(json, params)

Empty refinement object:
GoniometerRefinement with 0 geometries labeled: .


AttributeError: 'function' object has no attribute 'images_path'